resize selenium window?

In [ ]:

var rows = 6;
var columns = 6;
var screen = {
    height: 1024 * 6,
    width: 1350 * 6
};

function getScreenSize() {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

function resizeWindow(i = 0) {
    i = Math.abs(i + (typeof OFFSET !== 'undefined' ? OFFSET : 0));
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => typeof screen === 'undefined' ? getScreenSize() : screen)
        .catch(e => console.log(e))
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .catch(e => console.log(e))
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}

if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


only one window?



In [ ]:
var importer = require('../Core');

function closeAllTabs(keep) {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function closeAllWindows(keep) {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

function onlyOneWindow() {
    return client
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .catch(e => console.log(e))
    // TODO: close all tabs
}

module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');
var {getAllSessionUrls} = importer.import(['resize selenium window', 'get all session urls'], {client})

function openUrl(url) {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
}

function createNewWindows(urls) {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

function tileWindows(urls) {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .then(() => getAllSessionUrls())
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

$$.async();
// get multiple sets of 9 working for single page scraping in parallel
runSeleniumCell('tile chrome windows')
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



get all session and window urls?



In [ ]:
var importer = require('../Core');

function getAllSessionUrls(reposition = true) {
    var windowCounter = -1;
    var session = client.requestHandler.sessionID;
    return client
        .getActiveSessions()
        .then(r => {
            console.log(r);
            return importer.runAllPromises(r.map(s => resolve => {
                client.requestHandler.sessionID = s;
                return client
                    .getUrl()
                    .windowHandles()
                    .then(h => importer.runAllPromises(h.value.map(hwnd => resolve => {
                        windowCounter++
                        return client
                            .window(hwnd)
                            .then(() => reposition ? resizeWindow(windowCounter) : [])
                            .getUrl()
                            .then(r => resolve(r))
                            .catch(e => console.log(e))
                    })))
                    .then(r => resolve(r))
                    .catch(e => console.log(e))
            }))
        })
        .then(r => {
            client.requestHandler.sessionID = session;
            return r;
        })
        .catch(e => console.log(e))
}

module.exports = getAllSessionUrls;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async();
runSeleniumCell(['resize selenium window', 'get all session urls'], false)
    .then(r => r.getAllSessionUrls(false))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


send a joke?

In [12]:
var util = require('util');
var request = util.promisify(require('request'));

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i]));
    return co;
}

var jokes;
function getJoke() {
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => regexToArray(/^.*?Q:.*$|^.*?A:.*$/igm, res.body))
        .then(r => {
            r = r.reduce((arr, j, i) => {
                if(i % 2 === 1) {
                    arr.push([
                        r[i-1].replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, ''),
                        j.replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, '')
                    ]);
                }
                return arr;
            }, []);
            console.log(r);
            jokes = r;
            return r;
        })
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length);
            return arr[i];
        })
}

function sendJoke(hwnd) {
    var joke;
    return client
        .then(() => getJoke())
        .then(r => joke = r)
        .window(hwnd)
        .then(() => sendFacebookMessage(joke[0]))
        .pause(20000)
        .then(() => sendFacebookMessage(joke[1]))
}
module.exports = sendJoke;


[Function: sendJoke]

Monitor many chats at once and continuously


In [2]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var query = 'list of search engines';

var tileWindows, jokes, windows, sendFacebookMessage;

var friends = [
    
    
    /*

//    'https://www.facebook.com/messages/t/robert.holweger',
//    'https://www.facebook.com/messages/t/tintin.katy512',
//    'https://www.facebook.com/messages/t/greg.maserati.7',
//    'https://www.facebook.com/messages/t/scott.johnson.31149359',
//    'https://www.facebook.com/messages/t/railhunk',
//'https://www.facebook.com/messages/t/jeff.coffman.92',
//'https://www.facebook.com/messages/t/robert.holweger',
//'https://www.facebook.com/messages/t/100015169534875',
//'https://www.facebook.com/messages/t/chase.mcvey.9',
//'https://www.facebook.com/messages/t/Lloydster4',
//'https://www.facebook.com/messages/t/jessegill'
//'https://www.facebook.com/messages/t/lisa.potter.7140',
//'https://www.facebook.com/messages/t/StevenLockhart123',
//'https://www.facebook.com/messages/t/lidia.ws.7',
//'https://www.facebook.com/messages/t/corinne.cullinan',
//'https://www.facebook.com/messages/t/andreas.stylianou.98',


    
    'https://www.facebook.com/messages/t/ryan.raub',
    'https://www.facebook.com/messages/t/brian.harris.01',
    'https://www.facebook.com/messages/t/borikoni',
    'https://www.facebook.com/messages/t/tim.bangert.3',
    'https://www.facebook.com/messages/t/jsims730',
    'https://www.facebook.com/messages/t/Ryan-A-Stoffko-1784292651894577',
    'https://www.facebook.com/messages/t/sonu.karve.94',
    'https://www.facebook.com/messages/t/jason.hornack',
    'https://www.facebook.com/messages/t/BumpyJonson',
    'https://www.facebook.com/messages/t/frank.vitale01',
    'https://www.facebook.com/messages/t/jeff.kalina.79',
    'https://www.facebook.com/messages/t/demi.s.jiang',
    'https://www.facebook.com/messages/t/stephen.baier',
    'https://www.facebook.com/messages/t/mietusmm',
    'https://www.facebook.com/messages/t/chris.huie.14',
    'https://www.facebook.com/messages/t/vitali.kaush',
    'https://www.facebook.com/messages/t/FJoeAZ',
    'https://www.facebook.com/messages/t/SusanAgresta',
    'https://www.facebook.com/messages/t/andrew.s.mcvey.3',
    'https://www.facebook.com/messages/t/faulkner.shane',
    'https://www.facebook.com/messages/t/100013570236133',
    'https://www.facebook.com/messages/t/ryan.stoffko',
    'https://www.facebook.com/messages/t/6212130467169267713',
    'https://www.facebook.com/messages/t/joanna.street24',
    'https://www.facebook.com/messages/t/stephanie.desertgirl',
    'https://www.facebook.com/messages/t/vladimir.zuzukin',
    'https://www.facebook.com/messages/t/ScientificMaster.Me',
    'https://www.facebook.com/messages/t/maungkyarphyuu',
    'https://www.facebook.com/messages/t/maciejc',
    'https://www.facebook.com/messages/t/travis.hudson.507',
    'https://www.facebook.com/messages/t/ryan.raub',
    'https://www.facebook.com/messages/t/jennifer.reaves.58',
    'https://www.facebook.com/messages/t/jeff.kalina.79',
    'https://www.facebook.com/messages/t/talbert.tso',
    'https://www.facebook.com/messages/t/greg.maserati.7',
    'https://www.facebook.com/messages/t/katya.geissler',
    'https://www.facebook.com/messages/t/100008264283932',
    'https://www.facebook.com/messages/t/frederick.murphy.56',
    'https://www.facebook.com/messages/t/april.knapp.13',
    'https://www.facebook.com/messages/t/carisa.knight6000',
    'https://www.facebook.com/messages/t/Jakebot12',
    'https://www.facebook.com/messages/t/nathaniel.ellingson.3',
    'https://www.facebook.com/messages/t/4dumbdogs',
    'https://www.facebook.com/messages/t/marv.thornsberry',
    'https://www.facebook.com/messages/t/benjamin.maynard.9',
    'https://www.facebook.com/messages/t/enzo.armetta',
    'https://www.facebook.com/messages/t/MCChopper',
    'https://www.facebook.com/messages/t/suzi.abramsonjohnson',
    'https://www.facebook.com/messages/t/chris.paschall.37',
    'https://www.facebook.com/messages/t/tynababbit15',
    'https://www.facebook.com/messages/t/colleen.k.cullinan',
    'https://www.facebook.com/messages/t/ivana.vilotic.7',
    'https://www.facebook.com/messages/t/mark.camacho.50',
    'https://www.facebook.com/messages/t/fellchase2.0',
    'https://www.facebook.com/messages/t/mietusmm',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/100010177366596',
    'https://www.facebook.com/messages/t/julie.davis.792',
    'https://www.facebook.com/messages/t/tblondi',
    'https://www.facebook.com/messages/t/JesusAndTheAngels',
    'https://www.facebook.com/messages/t/callie.m.cullinan',
    'https://www.facebook.com/messages/t/Valorie.PONY.Teter',
    'https://www.facebook.com/messages/t/RogerRohweder',
    'https://www.facebook.com/messages/t/caudill.619',
    'https://www.facebook.com/messages/t/1623650947',
    'https://www.facebook.com/messages/t/cindy.huelskampjudy',
    'https://www.facebook.com/messages/t/100015169534875',
    'https://www.facebook.com/messages/t/brian.daviesjones.3',
    'https://www.facebook.com/messages/t/100005972775784',
    'https://www.facebook.com/messages/t/diane.weaver.948',
    'https://www.facebook.com/messages/t/jason.hornack',
    'https://www.facebook.com/messages/t/chase.mcvey.9',
    'https://www.facebook.com/messages/t/100006728686581',
    'https://www.facebook.com/messages/t/100013570236133',
    'https://www.facebook.com/messages/t/6212130467169267713',
    'https://www.facebook.com/messages/t/tim.bangert.3',
    'https://www.facebook.com/messages/t/katie.hiro',
    'https://www.facebook.com/messages/t/armando.moreno.90',
    'https://www.facebook.com/messages/t/john.kaniecki.3',
    'https://www.facebook.com/messages/t/chris.cullinan.792',
    'https://www.facebook.com/messages/t/ryan.stoffko',
    'https://www.facebook.com/messages/t/joanna.street24',
    'https://www.facebook.com/messages/t/stephanie.desertgirl',
    'https://www.facebook.com/messages/t/rohwedernt',
    'https://www.facebook.com/messages/t/david.earl.smith',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/teresa.cvach',
    'https://www.facebook.com/messages/t/firealwaysworks',
    'https://www.facebook.com/messages/t/crystaldgraziano',
    'https://www.facebook.com/messages/t/faulkner.shane',
    'https://www.facebook.com/messages/t/colleen.k.cullinan',
    'https://www.facebook.com/messages/t/lemcvey',
    'https://www.facebook.com/messages/t/james.cullinan.315',
    'https://www.facebook.com/messages/t/jaimeneufer',
    'https://www.facebook.com/messages/t/nathaniel.ellingson.3',
    'https://www.facebook.com/messages/t/aneesh.karve',
    'https://www.facebook.com/messages/t/robert.chandler',
    'https://www.facebook.com/messages/t/robert.holweger',
    'https://www.facebook.com/messages/t/keifer.street',
    'https://www.facebook.com/messages/t/mynameisjane',
    'https://www.facebook.com/messages/t/ragunr',
    'https://www.facebook.com/messages/t/johan.larson.56',
    'https://www.facebook.com/messages/t/abe.pralle',
    'https://www.facebook.com/messages/t/crystaldgraziano',
    'https://www.facebook.com/messages/t/wolfdieter.otte',
    'https://www.facebook.com/messages/t/carisa.knight6000',
    'https://www.facebook.com/messages/t/talbert.tso',
    'https://www.facebook.com/messages/t/abe.anwary',
    'https://www.facebook.com/messages/t/devin.west2',
    'https://www.facebook.com/messages/t/eurostar88',
    */
];

function tellJokes() {
    // get multiple sets of 9 working for single page scraping in parallel
    return runSeleniumCell([
        'tile chrome windows',
        'log in facebook',
        'send facebook message',
        'send a joke'
    ])
        .then(r => {
            tileWindows = r.tileWindows;
            sendFacebookMessage = r.sendFacebookMessage;
            sendJoke = r.sendJoke;
            return r.loginFacebook()
        })
        .then(() => tileWindows(friends))
        .then(windows => {
            return importer.runAllPromises(windows.value.map(hwnd => resolve => {
                return sendJoke(hwnd).then(() => resolve());
            }))
        })
        .catch(e => console.log(e))
}

$$.async();
tellJokes()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
[ 1510696908054, 'efc69a42acc1247d96fc1660cbab1e9c' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/efc69a42acc1247d96fc1660cbab1e9c/window_handle"
[ 1510696908062, '4396240da06751c50b39cf112ed569c2' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/4396240da06751c50b39cf112ed569c2/window_handle"
[ 1510696908066, '70c1f1e880ec2a185efa345e90b57b8f' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/70c1f1e880ec2a185efa345e90b57b8f/window_handle"
[ 1510696908071, '35b322dbfef60abc8536a98c271158fd' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/35b322dbfef60abc8536a98c271158fd/window_handle"
[ 1510696908076, 'bf7964e07ecc330968d1bbf9c9334693' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/bf7964e07ecc330968d1bbf9c9334693/window_handle"
[ 1510696908099, '912f5052a3261443593104ddea3f0274' ]
[16:00:49]  COMMAND	GET 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/window_handle"
[16:00:49]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/window"
[16:00:49

[16:01:18]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


[16:01:19]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


[16:01:19]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"
[16:01:40]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/element"
[16:01:40]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/element/0.1327179716975171-1/click"


[16:01:40]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


...

[16:04:23]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"
[16:04:43]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/element"
[16:04:43]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/element/0.7119447754970629-1/click"


[16:04:43]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


[16:04:43]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


[16:04:44]  COMMAND	POST 	 "/wd/hub/session/912f5052a3261443593104ddea3f0274/keys"


[ undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined,
  undefined ]

clean up old selenium sessions?

In [1]:
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
var runSeleniumCell = importer.import('selenium cell');
var createWebdriverClient = importer.import('webdriver client');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

var client, sessions;
function cleanUpSessions() {
    return execCmd('docker logs act-selenium')
        .then(r => regexToArray(/\/session\/(.+?)\//ig, r.join('\n'), 1))
        .then(r => sessions = r.filter((s, i, arr) => arr.indexOf(s) === i))
        .then(() => client = createWebdriverClient())
        .then(() => {
            return importer.runAllPromises(sessions.map(session => (resolve) => {
                client.requestHandler.sessionID = session;
                client
                    .windowHandle()
                    .then(r => client.window(r.value))
                    .session()
                    .end()
                    .then(s => resolve(s.sessionId))
                    .catch(e => resolve())
            }))
        })
}
module.exports = cleanUpSessions;


[Function: cleanUpSessions]

In [ ]:
$$.async()
cleanUpSessions();

Waiting xvfb...
Using installed websockify at /usr/bin/websockify
Starting webserver and WebSockets proxy on port 6080

04:29:03.689 INFO - Selenium build info: version: '3.5.0', revision: '8def36e068'
04:29:03.691 INFO - Launching a standalone Selenium Server
2017-11-10 04:29:03.734:INFO::main: Logging initialized @837ms to org.seleniumhq.jetty9.util.log.StdErrLog
04:29:03.836 INFO - Driver class not found: com.opera.core.systems.OperaDriver


Navigate to this URL:

    http://812c908edbad:6080/vnc.html?host=812c908edbad&port=6080

Press Ctrl-C to exit


04:29:03.894 INFO - Driver provider class org.openqa.selenium.ie.InternetExplorerDriver registration is skipped:
 registration capabilities Capabilities [{ensureCleanSession=true, browserName=internet explorer, version=, platform=WINDOWS}] does not match the current platform LINUX
04:29:03.894 INFO - Driver provider class org.openqa.selenium.edge.EdgeDriver registration is skipped:
 registration capabilities Capabilities [{browserName

sed: can't read /usr/profile/Default/Preferences: No such file or directory
sed: can't read /usr/profile/Default/Preferences: No such file or directory
Failed to read: session.ignoreBorder
Setting default value
Failed to read: session.forcePseudoTransparency
Setting default value
Failed to read: session.colorsPerChannel
Setting default value
Failed to read: session.doubleClickInterval
Setting default value
Failed to read: session.tabPadding
Setting default value
Failed to read: session.styleOverlay
Setting default value
Failed to read: session.slitlistFile
Setting default value
Failed to read: session.appsFile
Setting default value
Failed to read: session.tabsAttachArea
Setting default value
Failed to read: session.cacheLife
Setting default value
Failed to read: session.cacheMax
Setting default value
Failed to read: session.autoRaiseDelay
Setting default value
Failed to read: session.ignoreBorder
Setting default value
Failed to read: session.forcePseudoTransparency
Setting default valu

PORT=5900



10/11/2017 04:29:04   repaint the screen, also see the -fixscreen option for
10/11/2017 04:29:04   periodic repaints.
10/11/2017 04:29:04   Note: '-scale' is on and this can cause more problems.
10/11/2017 04:29:04 
10/11/2017 04:29:04 XFIXES available on display, resetting cursor mode
10/11/2017 04:29:04   to: '-cursor most'.
10/11/2017 04:29:04   to disable this behavior use: '-cursor arrow'
10/11/2017 04:29:04   or '-noxfixes'.
10/11/2017 04:29:04 using XFIXES for cursor drawing.
10/11/2017 04:29:04 GrabServer control via XTEST.
10/11/2017 04:29:04 
10/11/2017 04:29:04 Scroll Detection: -scrollcopyrect mode is in effect to
10/11/2017 04:29:04   use RECORD extension to try to detect scrolling windows
10/11/2017 04:29:04   (induced by either user keystroke or mouse input).
10/11/2017 04:29:04   If this yields undesired behavior (poor response, painting
10/11/2017 04:29:04   errors, etc) it may be disabled via: '-noscr'
10/11/2017 04:29:04   Also see the -help entry for tuning paramete

2017-11-10 04:48:00.019:INFO:osjshC.ROOT:qtp971848845-10: org.openqa.selenium.remote.server.WebDriverServlet-4678c730: Initialising WebDriverServlet
04:48:00.061 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@3cf6ad4f
04:48:00.063 INFO - /session/88484f1e0aa90e53f7c2c2d315a14401/window_handle: Executing GET on /session/88484f1e0aa90e53f7c2c2d315a14401/window_handle (handler: NoSessionHandler)
04:48:00.100 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@227d2c21
04:48:00.100 INFO - /session/5096e1c4d3d1e7e87910ef660b02b898/window_handle: Executing GET on /session/5096e1c4d3d1e7e87910ef660b02b898/window_handle (handler: NoSessionHandler)
04:48:00.111 INFO - Found handler: org.openqa.selenium.remote.server.commandhandler.NoSessionHandler@3946e62b
04:48:00.112 INFO - /session/14e96631161d2e2f1a86c86375095951/window_handle: Executing GET on /session/14e96631161d2e2f1a86c86375095951/window_handle (handler: NoSes